In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
import src.cleaning as clean
import seaborn as sns
import src.analysis.utils as utils
import numpy as np

In [ ]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

In [ ]:
#Restrict to only orders being sent not returned - being quantity > 0 
positive_quantity = cleaned_data.loc[cleaned_data['quantity']>0, :]

In [ ]:
customer_shipped_dates = positive_quantity.groupby('customer_id')['ship_date'].unique()

In [ ]:
customer_shipped_days_between = customer_shipped_dates.apply(utils.calculate_days_between)
customer_shipped_days_between

In [ ]:
customer_shipped_dates.apply(len).sum()

In [ ]:
average_customer_shipping_times = customer_shipped_days_between.dropna().apply(np.mean).sort_values()
average_customer_shipping_times[(average_customer_shipping_times > 0) & (average_customer_shipping_times <= 2)].shape

In [ ]:
shipping_difference_equal_1 = customer_shipped_days_between.apply(lambda row: utils.calculate_result_of_waiting(row, max_days_to_wait=1))
shipping_difference_equal_1.sum()

In [ ]:
# CHECKING
# If there is one shipment per customer as we wait for all their orders, this should be equal to number of customers
# Number of shipments - Number to subtract if we wait for all orders == Number of customers
customer_shipped_dates.apply(len).sum() - customer_shipped_days_between.apply(lambda row: utils.calculate_result_of_waiting(row, max_days_to_wait=12000)).sum() == positive_quantity['customer_id'].unique().shape[0]

In [ ]:
# Additional Checks
customer_shipped_dates[100000354]